### Step 1 – Load the Dataset


In [3]:
import pandas as pd
area_fire= pd.read_csv(r'working data\CA (cleaned+unit id) wildfire area data (1984 to 2023) .csv')
#weather_fire= pd.read_csv(r'CA_Weather (cleaned up) 1984-2023.csv')

Cleaning up the data, dropping extra column


In [4]:
area_fire.head()

#check the number of features and amount of entries
print('no. of data, no. of features:', area_fire.shape)
#check the value and type of each column(features)
print('data info:', area_fire.info())

area_fire= area_fire.dropna()


no. of data, no. of features: (22261, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22261 entries, 0 to 22260
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   _id            11200 non-null  float64
 1   ALARM_DATE     10618 non-null  object 
 2   CONT_DATE      8737 non-null   object 
 3   UNIT_ID        22195 non-null  object 
 4   GIS_ACRES      11200 non-null  float64
 5   Shape__Area    11200 non-null  float64
 6   Shape__Length  11200 non-null  float64
 7   YEAR_          11123 non-null  float64
dtypes: float64(5), object(3)
memory usage: 1.4+ MB
data info: None


tabula: convert pdf table into pandas df (failed coz it became a weird list, multiple values in one rows?)
tried using online pdf table to excel and import as xlsx

import numpy as np
#print(np.where(city['MACS-ID']== unitid)) -- not work due to differnt length

# Find values in unitid but not in city['MACS-ID']
missing_unitids = list(set(unitid) - set(city['MACS-ID']))

print(missing_unitids)

In [ ]:
#all the station mentioned in area_fire
unitid = list(area_fire["UNIT_ID"].unique())

agentcity= pd.read_excel(r'agency+unit id.xlsx',sheet_name='Table 5')

city_mentioned= agentcity[agentcity['Stored Value'].isin(unitid)]
#converting the mentioned fire station code to name
agent_name= city_mentioned['Display Value']
agent_name #all the mentioned fire station NAME



0                    Amador - El Dorado CAL FIRE
1                        Angeles National Forest
2      Bakersfield District - BLM (retired code)
4                        Beale Air Force Base FD
5          Bitter Creek National Wildlife Refuge
                         ...                    
108                Tuolumne - Calaveras CAL FIRE
109                           Vallejo Fire Dept.
110                    Vandenberg Air Force Base
111                               Ventura County
113                       Yosemite National Park
Name: Display Value, Length: 103, dtype: object

ca fire_agency-list is from 
https://firescope.caloes.ca.gov/Governance/Group%20Documents/MACS-ID-Agency-List.pdf

In [ ]:
##testing out if the fire station mentioned matches '"ca fire-Agency-List.xlsx" since it has city information
cafireagencywithcity= pd.read_excel(r'ca fire-Agency-List.xlsx')
cafireagencywithcity=cafireagencywithcity.dropna()
relevantmasc_id= cafireagencywithcity[cafireagencywithcity['MACS-ID'].isin(unitid)]
city_withfire= relevantmasc_id[['MACS-ID', 'CITY']]
city_withfire.head()
#the list of city we are gonna use to match with the weather stations
citymatchtoweather= relevantmasc_id['CITY']
citymatchtoweather


8               Camino
24             Arcadia
54      San Bernardino
55      San Bernardino
61            Monterey
             ...      
932            Visalia
953            Vallejo
955          Camarillo
967               Weed
1004          Yosemite
Name: CITY, Length: 89, dtype: object

for weather data: we have to align the weather station to the fire stations 
with their corresponding city


In [28]:
allweather_stn = pd.read_excel(r'ghcnd-stations-CA only.xlsx')
allweather_stn = allweather_stn.drop(columns=['GSN FLAG', 'HCN/CRN FLAG'])
allweather_stn.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,WMO ID
0,USC00040006,39.0333,-122.4333,601.1,CA,ABBOTT MINE,
1,USC00040010,38.2178,-121.2014,26.5,CA,ACAMPO 5NE,
2,USR0000CACT,34.4458,-118.2000,792.5,CA,ACTON CALIFORNIA,
3,USC00040014,34.4939,-118.2714,863.5,CA,ACTON ESCONDIDO FC261,
4,USC00040024,34.5833,-117.4167,869.0,CA,ADELANTO,


In [34]:
#matching the list of city to the weather stations to get their weather station id

weather_stnreq = allweather_stn[allweather_stn['NAME'].apply(lambda x: any(city.lower() in x.lower() for city in citymatchtoweather))]
weather_stnreq

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,WMO ID
39,USC00040161,41.4903,-120.5439,1332.9,CA,ALTURAS,
40,USC00040162,41.5147,-120.3642,1617.6,CA,ALTURAS 9 ENE,
41,USW00094299,41.4836,-120.5614,1335.9,CA,ALTURAS MUNI AP,
84,US1CALA0021,34.1617,-118.0285,198.7,CA,ARCADIA 2.1 NNE,
98,USW00024283,40.9783,-124.1047,64.3,CA,ARCATA EUREKA AP,
...,...,...,...,...,...,...,...
3122,USW00053150,37.7592,-119.8208,2017.8,CA,YOSEMITE VILLAGE 12 W,
3124,USC00049866,41.7025,-122.6417,826.6,CA,YREKA,
3125,US1CASK0005,41.7326,-122.6495,820.5,CA,YREKA 0.9 WNW,
3126,US1CASK0002,41.6651,-122.6208,895.2,CA,YREKA 4.5 S,


In [35]:
#list of weather station id we need to get from ftp
weatherid_list = weather_stnreq["ID   "]
weatherid_list

39      USC00040161
40      USC00040162
41      USW00094299
84      US1CALA0021
98      USW00024283
           ...     
3122    USW00053150
3124    USC00049866
3125    US1CASK0005
3126    US1CASK0002
3127    US1CASK0030
Name: ID   , Length: 413, dtype: object

In [36]:
import os
import requests

# Base URL
base_url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_station/"

# List of station IDs (modify as needed)


# Directory to save downloaded files
download_dir = "weather_data"
os.makedirs(download_dir, exist_ok=True)

for station_id in weatherid_list:
    file_name = f"{station_id}.csv.gz"  # NOAA files are in .csv.gz format
    file_url = base_url + file_name
    local_file_path = os.path.join(download_dir, file_name)

    # Download the file
    response = requests.get(file_url, stream=True)
    
    if response.status_code == 200:
        with open(local_file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {file_name} (Status Code: {response.status_code})")


Downloaded: USC00040161.csv.gz
Downloaded: USC00040162.csv.gz
Downloaded: USW00094299.csv.gz
Downloaded: US1CALA0021.csv.gz
Downloaded: USW00024283.csv.gz
Downloaded: USC00040383.csv.gz
Downloaded: US1CAPC0055.csv.gz
Downloaded: US1CAPC0054.csv.gz
Downloaded: US1CAPC0012.csv.gz
Downloaded: US1CAPC0037.csv.gz
Downloaded: US1CAPC0032.csv.gz
Downloaded: USC00040385.csv.gz
Downloaded: USC00040439.csv.gz
Downloaded: USC00040444.csv.gz
Downloaded: US1CAKN0022.csv.gz
Downloaded: USW00023155.csv.gz
Downloaded: USC00040519.csv.gz
Downloaded: USC00040521.csv.gz
Downloaded: US1CASR0023.csv.gz
Downloaded: US1CASR0025.csv.gz
Downloaded: US1CASR0035.csv.gz
Downloaded: USW00023161.csv.gz
Downloaded: USC00040823.csv.gz
Downloaded: US1CAIN0001.csv.gz
Downloaded: US1CAIN0013.csv.gz
Downloaded: US1CAIN0007.csv.gz
Downloaded: US1CAIN0004.csv.gz
Downloaded: US1CAIN0012.csv.gz
Downloaded: US1CAIN0010.csv.gz
Downloaded: US1CAIN0002.csv.gz
Downloaded: US1CAIN0005.csv.gz
Downloaded: USW00023157.csv.gz
Download

In [7]:
area_fire=area_fire.drop(['CONT_DATE'],axis=1)


In [8]:

# Join and drop unmatched values
df = df.join(scores, how='inner')  # 'inner' keeps only matching rows

print(df)

NameError: name 'df' is not defined

In [ ]:
area_date=area_fire.iloc[:, 0:4]
area_date.set_index(['ALARM_DATE'])
weather_fire.set_index(['DATE'])


,FIRE_START_DAY,PRECIPITATION,MAX_TEMP,MIN_TEMP,AVG_WIND_SPEED,TEMP_RANGE,WIND_TEMP_RATIO,LAGGED_PRECIPITATION,LAGGED_AVG_WIND_SPEED,SEASON,DAY_OF_YEAR
DATE,,,,,,,,,,,
31/12/2023,False,0.00,60,51,4.92,9,0.082000,0.62,5.558571,Winter,365
30/12/2023,False,0.62,62,54,8.50,8,0.137097,0.62,5.558571,Winter,364
29/12/2023,False,0.00,65,52,3.80,13,0.058462,0.00,5.175714,Winter,363
28/12/2023,False,0.00,64,50,5.59,14,0.087344,0.50,5.782857,Winter,362
27/12/2023,False,0.00,62,52,5.59,10,0.090161,1.83,6.198571,Winter,361
...,...,...,...,...,...,...,...,...,...,...,...
5/1/1984,False,0.00,74,49,5.14,25,0.069459,0.00,5.100000,Winter,5
4/1/1984,False,0.00,76,45,4.70,31,0.061842,0.00,5.090000,Winter,4
3/1/1984,False,0.00,70,47,5.37,23,0.076714,0.00,5.220000,Winter,3


### Step 2 – Preprocess the Dataset
 
 The features are all on different scales. To bring them all to a common scale, we’ll use the **StandardScaler** that transforms the features to have zero mean and unit variance:

##### Thoughts
run a PCA to see which top 2-3 factors are the most crucial?

In [ ]:
import sklearn 
from sklearn.preprocessing import StandardScaler
#separate features and
features = ['sepal length', 'sepal width', 'petal length', 'petal width']
std_scaler = StandardScaler()
scaled_cafire = std_scaler.fit_transform(ca_fire)

NameError: name 'ca_fire' is not defined

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split